In [1]:
import csv
import numpy as np
import pandas as pd
import os

In [2]:
# File Path (please update this part according to the location)
path = '...'

# Step 1. Read and convert data into csv

In [3]:
# ASD data

path_fmri_asd = os.chdir("{}/asd".format(path))
fmri_asd= os.listdir(path_fmri_asd)

fmri_asd_values = []
num_features = 116 ## AAL 116 ROI


for file in fmri_asd:
    each_file = []
    if file.endswith(".txt"):
        with open(file, newline='') as csvfile:
            data = csv.reader(csvfile, delimiter = ' ')
            data_array = []
            for d in data:
                float_d = []
                for num in d:
                    #print(num)
                    float_d.append((float(num)))
                data_array.append(float_d)
            data_array = np.array(data_array).reshape(num_features, num_features)
            for i in range(num_features):
                for j in range(i + 1, num_features):
                    each_file.append(data_array[i][j])
        fmri_asd_values.append(each_file)

header = []
for i in range(num_features):
    for j in range(i + 1, num_features):
        header.append((i + 1, j + 1))
        
ASD_data = pd.DataFrame(fmri_asd_values, columns = header)

ASD_data.to_csv("{}/ASD_data_values.csv".format(path), index=False) # Update your path accordingly

In [4]:
# TD data

path_fmri_Typical_Development = os.chdir("{}/td/".format(path))
fmri_Typical_Development = os.listdir(path_fmri_Typical_Development)

fmri_Typical_Development_values = []
num_features = 116


for file in fmri_Typical_Development:
    each_file = []
    if file.endswith(".txt"):
        with open(file, newline='') as csvfile:
            data = csv.reader(csvfile, delimiter = ' ')
            data_array = []
            for d in data:
                float_d = []
                for num in d:
                    #print(num)
                    float_d.append((float(num)))
                data_array.append(float_d)
            data_array = np.array(data_array).reshape(num_features, num_features)
            for i in range(num_features):
                for j in range(i + 1, num_features):
                    each_file.append(data_array[i][j])
        fmri_Typical_Development_values.append(each_file)

header = []
for i in range(num_features):
    for j in range(i + 1, num_features):
        header.append((i + 1, j + 1))
        

TD_data = pd.DataFrame(fmri_Typical_Development_values, columns = header)
#print(fmri_Typical_Development_values_fr)

TD_data.to_csv("{}/TD_data_values.csv".format(path), index=False) # Update your path accordingly


# Step 2. Two-sample t-test for signficant edges identification

In [5]:
import scipy.stats

ASD_data = pd.read_csv("{}/ASD_data_values.csv".format(path))
TD_data = pd.read_csv("{}/TD_data_values.csv".format(path))

columns = (ASD_data.columns)
ttest_stat = []
ttest_pval = []

for i in columns:
    ASD_region_i = np.array(ASD_data[i]).reshape(-1,1)
    TD_region_i = np.array(TD_data[i]).reshape(-1,1)
    t_stat, t_pval = scipy.stats.ttest_ind(ASD_region_i, TD_region_i, equal_var = False)
    ttest_stat.append(float(t_stat))
    ttest_pval.append(float(t_pval))
    
regions_fr = pd.DataFrame(columns, columns=['Regions Connectivity'])
ttest_stat_fr = pd.DataFrame(ttest_stat, columns=['T statistics'])
ttest_pval_fr = pd.DataFrame(ttest_pval, columns=['P-values'])
print(regions_fr)

region_1 = []
region_2 = []


for i in range(len(regions_fr)):
    regions = (regions_fr.iloc[i])
    a, b = regions[0][1:-1].split(",")
    a, b = int(a), int(b)
    region_1.append(a)
    region_2.append(b)
    
regions_1 = pd.DataFrame(region_1, columns=['Region 1'])
regions_2 = pd.DataFrame(region_2, columns=['Region 2'])

t_test_results = pd.concat([regions_1,regions_2, ttest_stat_fr, ttest_pval_fr], axis = 1)
#print(t_test_results)

sign_results = t_test_results[t_test_results['P-values'] <= 0.05]
#print(sign_results)

     Regions Connectivity
0                  (1, 2)
1                  (1, 3)
2                  (1, 4)
3                  (1, 5)
4                  (1, 6)
...                   ...
6665           (113, 115)
6666           (113, 116)
6667           (114, 115)
6668           (114, 116)
6669           (115, 116)

[6670 rows x 1 columns]


# Step 3. Construct subgraphs by using edges from each cluster

In [6]:
from collections import defaultdict
from itertools import combinations

In [7]:
# Determine the clusters

sign_edges_list = sign_results
quantiles = list((abs(sign_edges_list['T statistics'])).quantile([.20, .40, .60, .80]))

z_1 = sign_edges_list[(abs(sign_edges_list['T statistics']) < quantiles[0])]
z_2 = sign_edges_list[(abs(sign_edges_list['T statistics'])>= quantiles[0]) & (abs(sign_edges_list['T statistics']) < quantiles[1])]
z_3 = sign_edges_list[(abs(sign_edges_list['T statistics'])>= quantiles[1])& (abs(sign_edges_list['T statistics']) < quantiles[2])]
z_4 = sign_edges_list[(abs(sign_edges_list['T statistics'])>= quantiles[2])& (abs(sign_edges_list['T statistics']) < quantiles[3])]
z_5 = sign_edges_list[(abs(sign_edges_list['T statistics'])>= quantiles[3])]

print("Number of edges in Cluster 1:",len(z_1))
print("Number of edges in Cluster 2:",len(z_2))
print("Number of edges in Cluster 3:",len(z_3))
print("Number of edges in Cluster 4:",len(z_4))
print("Number of edges in Cluster 5:",len(z_5))


Number of edges in Cluster 1: 52
Number of edges in Cluster 2: 79
Number of edges in Cluster 3: 66
Number of edges in Cluster 4: 84
Number of edges in Cluster 5: 80


In [8]:
# Group significant edges into clusters

cluster_edges_list = []
for i in range(1,6):
    cluster_name = f'z_{i}'
    cluster = eval(cluster_name)
    cluster_edge = []
    if len(cluster) != 0:
        for j in range(len(cluster)):
            region_1, region_2 = cluster['Region 1'].iloc[j], cluster['Region 2'].iloc[j]
            cluster_edge.append([region_1, region_2])
        cluster_edges_list.append(cluster_edge)



qualified_sign_edges = []

for i in range(len(sign_edges_list)):
    region_1, region_2 = sign_edges_list['Region 1'].iloc[i], sign_edges_list['Region 2'].iloc[i]
    qualified_sign_edges.append((region_1, region_2))
    

In [9]:
# The functions below are to generate the subgraphs using signficant edges in each cluster, and remove those duplicates

def generate_all_cycles(nodes, edges):
    def dfs(node, start, path, visited):
        if node == start and path and len(path) > 2:
            cycle = tuple(sorted(set(path)))
            if cycle not in all_cycles:  # Check for duplicates
                all_cycles.add(cycle)
            return
        if visited[node]:
            return
        visited[node] = True
        path.append(node)
        for neighbor in graph[node]:
            if not visited[neighbor] or (neighbor == start and len(path) > 1):
                dfs(neighbor, start, path, visited.copy())
        path.pop()

    # Convert edge list to adjacency list
    graph = {node: set() for node in nodes}
    for u, v in edges:
        graph[u].add(v)
        graph[v].add(u)

    all_cycles = set()
    for node in nodes:
        dfs(node, node, [], {n: False for n in nodes})

    return [list(cycle) for cycle in all_cycles]

def remove_subcycles(cycles):
    unique_cycles = []
    for cycle in sorted(cycles, key=len, reverse=True):
        if not any(set(cycle).issubset(set(other_cycle)) and len(cycle) < len(other_cycle) for other_cycle in unique_cycles):
            unique_cycles.append(cycle)
    return unique_cycles

In [10]:
# AAL has 116 ROIs
nodes = []

for i in range(1,117):
    nodes.append(i)
    

# Here we need to check subgraphs constructed in each cluster
cluster_index = []

ordered_unique_cycles = []
ordered_unique_cycle_edges = []

for i in range(len(cluster_edges_list)):
    edges = cluster_edges_list[i]    
    #Generate all subgraphs
    all_cycles = generate_all_cycles(nodes, edges)
    #Remove duplicates
    unique_cycle = remove_subcycles(all_cycles)
    print("Cluster {}".format(i + 1))
    print("Number of subgraphs:", len(unique_cycle))
    if len(unique_cycle) != 0:
        for j in range(len(unique_cycle)):
            cluster_index.append(i + 1)
        ordered_unique_cycle_edges.append(edges)
        ordered_unique_cycles.append(unique_cycle)
    print("\n")



Cluster 1
Number of subgraphs: 0


Cluster 2
Number of subgraphs: 2


Cluster 3
Number of subgraphs: 2


Cluster 4
Number of subgraphs: 3


Cluster 5
Number of subgraphs: 17




In [11]:
# This function shows how exactly the subgraphs look like (e.g., which node is connected to which one)

def reorder_ring_according_to_edges(nodes, edges):
    def dfs(node, path):
        if len(path) == len(nodes):
            if path[0] in graph[path[-1]]:  # Check if we can complete the cycle
                return path + [path[0]]
            else:
                return None

        for neighbor in graph[node]:
            if neighbor not in path:
                new_path = dfs(neighbor, path + [neighbor])
                if new_path:
                    return new_path

        return None

    # Convert edge list to adjacency list
    graph = {node: set() for node in nodes}
    for u, v in edges:
        if u in nodes and v in nodes:  # Only consider edges within the specified nodes
            graph[u].add(v)
            graph[v].add(u)

    # Attempt to reorder from each node
    for start_node in nodes:
        path = dfs(start_node, [start_node])
        if path:
            return path

    return "No valid reordering found."


reordered_rings = []
for i in range(len(ordered_unique_cycles)):
    cycle = ordered_unique_cycles[i]
    edges = ordered_unique_cycle_edges[i]
    for node in cycle:
        reordered_ring = reorder_ring_according_to_edges(node, edges)
        reordered_rings.append(reordered_ring)
        
reordered_rings_info = []

for i in range(len(reordered_rings)):
    print("Graph {}".format(i + 1))
    print(reordered_rings[i])
    reordered_rings_info.append(reordered_rings[i][:-1])
    print("\t")


Graph 1
[10, 16, 63, 45, 95, 82, 111, 71, 17, 11, 67, 46, 78, 31, 44, 24, 101, 35, 10]
	
Graph 2
[36, 64, 37, 70, 89, 111, 91, 36]
	
Graph 3
[23, 26, 51, 23]
	
Graph 4
[12, 96, 28, 12]
	
Graph 5
[5, 57, 86, 104, 34, 35, 58, 92, 68, 8, 99, 97, 69, 17, 12, 114, 26, 73, 10, 27, 15, 6, 102, 63, 75, 42, 74, 65, 72, 88, 45, 7, 5]
	
Graph 6
[5, 57, 86, 104, 109, 82, 97, 69, 17, 12, 114, 26, 73, 10, 27, 15, 6, 102, 63, 75, 42, 74, 65, 72, 88, 45, 7, 5]
	
Graph 7
[8, 99, 97, 82, 109, 104, 34, 35, 58, 92, 68, 8]
	
Graph 8
[4, 75, 29, 41, 77, 74, 80, 24, 9, 102, 51, 114, 15, 43, 11, 110, 46, 85, 47, 81, 61, 14, 52, 4]
	
Graph 9
[4, 75, 29, 41, 77, 74, 80, 21, 22, 114, 15, 43, 11, 110, 46, 85, 47, 81, 61, 14, 52, 4]
	
Graph 10
[4, 24, 80, 21, 22, 114, 15, 43, 11, 110, 46, 85, 47, 81, 57, 18, 74, 77, 41, 29, 75, 4]
	
Graph 11
[4, 24, 9, 102, 51, 114, 15, 43, 11, 110, 46, 85, 47, 81, 57, 18, 74, 77, 41, 29, 75, 4]
	
Graph 12
[4, 75, 29, 41, 77, 74, 80, 21, 22, 114, 51, 102, 9, 46, 85, 47, 81, 61, 14

In [12]:
filtered_cycles = []


for cycle_list in (ordered_unique_cycles):
    for cycle in cycle_list:
        filtered_cycles.append(list(cycle))

filtered_cycles_fr = pd.DataFrame((filtered_cycles)).astype("Int32")
cluster_index_fr = pd.DataFrame(cluster_index, columns = ['Cluster'])
filtered_cycles_with_clusters = pd.concat([cluster_index_fr, filtered_cycles_fr], axis = 1)
filtered_cycles_fr.to_csv("{}/ring_nodes_5_clusters.csv".format(path),index = False)

original_list = cluster_edges_list

# Converting each inner list to a tuple
modified_list = [[tuple(inner_list) for inner_list in outer_list] for outer_list in original_list]

loop_edge = []

for idx in range(len(filtered_cycles_with_clusters)):
    cycle_row = filtered_cycles_with_clusters.iloc[idx]
    cluster = cycle_row['Cluster'] - 1
    cycle = list((cycle_row.iloc[1:]).dropna())
    combines_two = list(combinations(cycle, 2))
    each_cycle_sign_edge = []
    for comb in combines_two:
        if comb in modified_list[cluster]:
            each_cycle_sign_edge.append(comb)            
    loop_edge.append(each_cycle_sign_edge)

In [13]:
loop_edge_fr = pd.DataFrame(loop_edge)
loop_edge_fr.to_csv("{}/ring_topology_5_clusters.csv".format(path), index=False)

# Step 4. Extract Eigen-Entropies from all subgraphs

In [14]:
from math import log

def eigen_entropy(correlation_array):
    w,v = np.linalg.eig((correlation_array))
    w = np.real_if_close(w,tol=1)
    w_abs = abs(w)
    w_sum = np.sum(w_abs)
    ent_int = []
    
    for i in w_abs:
        if i ==0:
            ent_i = 0
        else:
            ent_i = -(i/w_sum)*(log(i/w_sum))
        ent_int.append(ent_i)
    entropy_initial = np.sum(ent_int)
    entropy_initial = round(entropy_initial,4)
    return entropy_initial

In [15]:
ASD_data = pd.read_csv("{}/ASD_data_values.csv".format(path))
Control_data = pd.read_csv("{}/TD_data_values.csv".format(path))

ring_topology_lists = pd.read_csv("{}/ring_topology_5_clusters.csv".format(path))
ring_nodes = pd.read_csv("{}/ring_nodes_5_clusters.csv".format(path))


In [16]:
def create_adjacency_matrix(nodes, edge_strings, edge_values):
    # Initialize a matrix with zeros
    adj_matrix = np.zeros((len(nodes), len(nodes)))

    # Set diagonal elements to 1
    np.fill_diagonal(adj_matrix, 1)

    # Check if the length of edge_strings and edge_values are the same
    if len(edge_strings) != len(edge_values):
        raise ValueError("The length of edge_strings and edge_values must be the same")

    # Parse and add edges with values
    for edge_str, value in zip(edge_strings, edge_values):
        # Extract nodes from the edge string
        node1, node2 = map(int, edge_str.strip('()').split(', '))
        # Adjust indices to match matrix indexing (starting from 0)
        index1, index2 = nodes.index(node1), nodes.index(node2)
        # Set the corresponding positions in the matrix to the given edge value
        adj_matrix[index1][index2] = adj_matrix[index2][index1] = value

    return adj_matrix

In [17]:
ASD_ee_all = np.empty((len(ASD_data),0))
Control_ee_all = np.empty((len(Control_data),0))

ring_index = 0

for i in range(len(ring_topology_lists)):
    ring_index += 1
    topology_list = list(ring_topology_lists.iloc[i])
    topology_list_clear = [item for item in topology_list if not pd.isna(item) ]

    node_list = list(ring_nodes.iloc[i])
    node_list_clear = [int(item) for item in node_list if not pd.isna(item) ]
    
    
    ASD_ee_each_ring = []
    
    ASD_data_each_topology = np.array(ASD_data[topology_list_clear])
    for j in range(len(ASD_data_each_topology)):
        ASD_data_sample_values = list(ASD_data_each_topology[j])
        ASD_sample_matrix = create_adjacency_matrix(node_list_clear,topology_list_clear, ASD_data_sample_values)
        ASD_ee_sample = eigen_entropy(ASD_sample_matrix)
        ASD_ee_each_ring.append(ASD_ee_sample)
    
    ASD_ee_each_ring_np = np.array(ASD_ee_each_ring).reshape(-1,1)
    
    ASD_ee_all = np.concatenate((ASD_ee_all,ASD_ee_each_ring_np),axis = 1)
        
    Control_ee_each_ring = []
    
    Control_data_each_topology = np.array(Control_data[topology_list_clear])
    for j in range(len(Control_data_each_topology)):
        Control_data_sample_values = list(Control_data_each_topology[j])
        Control_sample_matrix = create_adjacency_matrix(node_list_clear,topology_list_clear, Control_data_sample_values)
        Control_ee_sample = eigen_entropy(Control_sample_matrix)
        Control_ee_each_ring.append(Control_ee_sample)
    
    Control_ee_each_ring_np = np.array(Control_ee_each_ring).reshape(-1,1)
    Control_ee_all = np.concatenate((Control_ee_all,Control_ee_each_ring_np),axis = 1)  


In [18]:
EE_features_names = []

for i in range(1, len(ASD_ee_all[0])+1):
    EE_features_names.append("EE(S{})".format(i))


ASD_ee_all_fr = pd.DataFrame(ASD_ee_all, columns = EE_features_names)
Control_ee_all_fr = pd.DataFrame(Control_ee_all, columns = EE_features_names)

ASD_ee_all_fr.to_csv("{}/ASD_node_nets_5_clusters.csv".format(path), index=False)
Control_ee_all_fr.to_csv("{}/Control_node_nets_5_clusters.csv".format(path), index=False)

# Step 5. Assessment by Machine Learning (KNN classifier)

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

In [20]:
ASD_data = pd.read_csv("{}/ASD_node_nets_5_clusters.csv".format(path))
ASD_label = []

for i in range(len(ASD_data)):
    ASD_label.append(1)

    
Control_data = pd.read_csv("{}/Control_node_nets_5_clusters.csv".format(path))
Control_label = []

for i in range(len(Control_data)):
    Control_label.append(0)


In [21]:
ASD_data = np.array(ASD_data)
ASD_label = np.array(ASD_label).reshape(-1,1)


Control_data = np.array(Control_data)
Control_label = np.array(Control_label).reshape(-1,1)


data = np.concatenate((ASD_data, Control_data), axis = 0)
labels = np.concatenate((ASD_label, Control_label), axis = 0)



In [22]:
neighbors = []

for i in range(3, 31):
    neighbors.append(i)

X_train_std = data
y_train = labels

knn = KNeighborsClassifier()

clf = GridSearchCV(knn, cv = 5, param_grid = {'n_neighbors': neighbors})
clf.fit(data,labels.ravel())
print("Best parameters:",clf.best_params_['n_neighbors'])


Best parameters: 20


In [23]:
np.random.seed(42)
from collections import Counter

X_train = data
y_train = labels


knn = KNeighborsClassifier(n_neighbors = clf.best_params_['n_neighbors'] )

kf = KFold(n_splits=5, random_state = 42, shuffle=True)
all_accuracy = []

for i, (train_index, val_index) in enumerate(kf.split(X_train)):
    data_cv_train, data_cv_val = X_train[train_index], X_train[val_index]
    label_cv_train, label_cv_val = y_train[train_index], y_train[val_index]
    knn.fit(data_cv_train, label_cv_train.ravel())
    y_train_pred = knn.predict(data_cv_train)
    acc_train = accuracy_score(label_cv_train, y_train_pred)
    y_val_pred = knn.predict(data_cv_val)
    acc_val = accuracy_score(label_cv_val, y_val_pred)
    all_accuracy.append(acc_val)
    
    
    
print("Average Accuracy:",round(np.mean(all_accuracy),2))

Average Accuracy: 0.71
